<a href="https://colab.research.google.com/github/zhannatoleubek-png/special-okx-chainsaw/blob/main/%D0%94%D0%B8%D0%BF%D1%81%D0%B8%D0%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import requests
import warnings
import time
import hmac
import hashlib
import base64
from datetime import datetime, timedelta
warnings.filterwarnings('ignore')

class OKXClient:
    def __init__(self):
        self.base_url = "https://www.okx.com"
        self.public_endpoints = {
            'candles': '/api/v5/market/candles',
            'ticker': '/api/v5/market/ticker',
            'instruments': '/api/v5/public/instruments'
        }

    def get_candles(self, symbol='BTC-USDT', timeframe='5m', limit=300):
        """Получение свечных данных с OKX"""
        try:
            # Конвертация таймфреймов в формат OKX
            tf_mapping = {
                '1m': '1m', '3m': '3m', '5m': '5m', '15m': '15m',
                '30m': '30m', '1h': '1H', '4h': '4H', '1d': '1D'
            }

            okx_tf = tf_mapping.get(timeframe, '5m')
            inst_id = symbol.replace('-', '-')  # BTC-USDT -> BTC-USDT

            url = f"{self.base_url}{self.public_endpoints['candles']}"
            params = {
                'instId': inst_id,
                'bar': okx_tf,
                'limit': limit
            }

            response = requests.get(url, params=params, timeout=10)
            data = response.json()

            if data['code'] == '0':
                candles = data['data']
                # OKX возвращает данные в обратном порядке (новые первыми)
                candles.reverse()

                df_data = []
                for candle in candles:
                    df_data.append({
                        'Timestamp': datetime.fromtimestamp(int(candle[0]) / 1000),
                        'Open': float(candle[1]),
                        'High': float(candle[2]),
                        'Low': float(candle[3]),
                        'Close': float(candle[4]),
                        'Volume': float(candle[5]),
                        'VolumeCcy': float(candle[6])  # Объем в валюте
                    })

                df = pd.DataFrame(df_data)
                df.set_index('Timestamp', inplace=True)
                return df
            else:
                print(f"Ошибка OKX: {data['msg']}")
                return self._create_demo_data()

        except Exception as e:
            print(f"Ошибка получения данных с OKX: {e}")
            return self._create_demo_data()

    def get_ticker(self, symbol='BTC-USDT'):
        """Получение текущей цены"""
        try:
            inst_id = symbol.replace('-', '-')
            url = f"{self.base_url}{self.public_endpoints['ticker']}"
            params = {'instId': inst_id}

            response = requests.get(url, params=params, timeout=5)
            data = response.json()

            if data['code'] == '0':
                return float(data['data'][0]['last'])
            else:
                return 50000.0
        except:
            return 50000.0

    def _create_demo_data(self):
        """Создание демо-данных при ошибке"""
        print("Создание демо-данных...")
        dates = pd.date_range(start=datetime.now() - timedelta(days=7),
                             periods=500, freq='5min')
        np.random.seed(42)
        prices = 50000 + np.cumsum(np.random.randn(500) * 100)

        df = pd.DataFrame({
            'Open': prices + np.random.randn(500) * 50,
            'High': prices + np.abs(np.random.randn(500) * 100),
            'Low': prices - np.abs(np.random.randn(500) * 100),
            'Close': prices,
            'Volume': np.random.randint(1000, 10000, 500),
            'VolumeCcy': np.random.randint(50000, 500000, 500)
        }, index=dates)
        return df

class TelegramBot:
    def __init__(self, bot_token, chat_id):
        self.bot_token = bot_token
        self.chat_id = chat_id
        self.base_url = f"https://api.telegram.org/bot{bot_token}/"

    def send_message(self, text):
        """Отправка сообщения в Telegram"""
        url = self.base_url + "sendMessage"
        payload = {
            'chat_id': self.chat_id,
            'text': text,
            'parse_mode': 'HTML'
        }
        try:
            response = requests.post(url, data=payload, timeout=10)
            return response.json()
        except Exception as e:
            print(f"Ошибка отправки в Telegram: {e}")
            return None

    def send_signal(self, signal_data):
        """Отправка торгового сигнала"""
        emoji = "🟢" if signal_data['signal_type'] == "BUY" else "🔴"

        message = f"""
{emoji} <b>СКАЛЬПИНГ СИГНАЛ OKX</b> {emoji}

🎯 <b>Тип:</b> {signal_data['signal_type']}
💰 <b>Цена:</b> ${signal_data['price']:.2f}
💪 <b>Сила сигнала:</b> {signal_data['strength']}/5
📊 <b>Таймфрейм:</b> {signal_data['timeframe']}

<b>📈 МУЛЬТИТАЙМФРЕЙМ АНАЛИЗ:</b>
{signal_data['timeframe_analysis']}

<b>🏦 ДОМИНАЦИЯ BTC:</b> {signal_data['btc_dominance']:.1f}%

<b>🎯 УРОВНИ:</b>
🛡️ <b>Stop-Loss:</b> ${signal_data['stop_loss']:.2f}
🎯 <b>Take-Profit:</b> ${signal_data['take_profit']:.2f}
📈 <b>R/R:</b> 1:1.7

<b>📊 ПОДТВЕРЖДЕНИЯ:</b>
{signal_data['indicators']}

<b>⚡ ИСТОЧНИК:</b> OKX Spot
⏰ <b>Время:</b> {datetime.now().strftime('%H:%M:%S')}
        """

        return self.send_message(message)

class OKXMultiTimeframeAnalyzer:
    def __init__(self):
        self.okx_client = OKXClient()
        self.timeframes = {
            'D1': '1d',
            'H4': '4h',
            'H1': '1h',
            'M30': '30m',
            'M15': '15m',
            'M5': '5m'
        }

    def get_btc_dominance(self):
        """Получение доминации Bitcoin"""
        try:
            url = "https://api.coingecko.com/api/v3/global"
            response = requests.get(url, timeout=10)
            data = response.json()
            return data['data']['market_cap_percentage']['btc']
        except:
            return 52.5

    def analyze_timeframe(self, data, timeframe_name):
        """Анализ одного таймфрейма"""
        if len(data) < 20:
            return f"{timeframe_name}: ⚪ НЕТ ДАННЫХ"

        try:
            current_price = float(data['Close'].iloc[-1])
            prev_price = float(data['Close'].iloc[-2]) if len(data) > 1 else current_price

            # Трендовые индикаторы
            ema_20 = float(data['Close'].ewm(span=20).mean().iloc[-1])
            ema_50 = float(data['Close'].ewm(span=50).mean().iloc[-1])

            # RSI
            delta = data['Close'].diff()
            gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
            loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
            rs = gain / loss
            rsi = float(100 - (100 / (1 + rs)).iloc[-1])

            # Определение тренда
            if ema_20 > ema_50 and current_price > ema_20:
                trend = "🟢 BULLISH"
            elif ema_20 < ema_50 and current_price < ema_20:
                trend = "🔴 BEARISH"
            else:
                trend = "🟡 NEUTRAL"

            # Изменение цены
            price_change = ((current_price - prev_price) / prev_price) * 100 if prev_price != 0 else 0

            # Уровни RSI
            if rsi > 70:
                rsi_status = "🔴 ПЕРЕКУПЛЕН"
            elif rsi < 30:
                rsi_status = "🟢 ПЕРЕПРОДАН"
            else:
                rsi_status = "⚪ НОРМА"

            return f"{timeframe_name}: {trend} | RSI: {rsi:.1f} ({rsi_status}) | Изм: {price_change:+.2f}%"

        except Exception as e:
            return f"{timeframe_name}: ❌ ОШИБКА"

    def get_multi_timeframe_analysis(self, symbol="BTC-USDT"):
        """Анализ по всем таймфреймам"""
        analysis = []
        dominance = self.get_btc_dominance()

        for tf_name, tf_interval in self.timeframes.items():
            try:
                data = self.okx_client.get_candles(symbol, tf_interval, 100)
                if not data.empty and len(data) > 20:
                    tf_analysis = self.analyze_timeframe(data, tf_name)
                    analysis.append(tf_analysis)
                else:
                    analysis.append(f"{tf_name}: ⚪ НЕТ ДАННЫХ")
            except Exception as e:
                analysis.append(f"{tf_name}: ❌ ОШИБКА")

        return "\n".join(analysis), dominance

class OKXScalpingAlgorithm:
    def __init__(self, telegram_bot=None):
        self.telegram_bot = telegram_bot
        self.okx_client = OKXClient()
        self.timeframe_analyzer = OKXMultiTimeframeAnalyzer()
        self.last_signal_time = None

        # Оптимизированные параметры для скальпинга на OKX
        self.parameters = {
            'ema_fast': 5,
            'ema_slow': 13,
            'rsi_period': 11,
            'volume_ma': 8,
            'atr_period': 7
        }

    def calculate_advanced_indicators(self, df):
        """Расчет улучшенных индикаторов"""
        p = self.parameters

        # Убедимся, что все данные в правильном формате
        for col in ['Open', 'High', 'Low', 'Close', 'Volume']:
            df[col] = pd.to_numeric(df[col], errors='coerce')

        # Быстрые EMA для скальпинга
        df['ema_fast'] = df['Close'].ewm(span=p['ema_fast']).mean()
        df['ema_slow'] = df['Close'].ewm(span=p['ema_slow']).mean()

        # VWAP для внутридневной торговли
        typical_price = (df['High'] + df['Low'] + df['Close']) / 3
        df['vwap'] = (typical_price * df['Volume']).cumsum() / df['Volume'].cumsum()

        # Оптимизированный RSI
        delta = df['Close'].diff()
        gain = (delta.where(delta > 0, 0)).ewm(span=p['rsi_period']).mean()
        loss = (-delta.where(delta < 0, 0)).ewm(span=p['rsi_period']).mean()
        rs = gain / loss
        df['rsi'] = 100 - (100 / (1 + rs))

        # Stochastic RSI для скальпинга
        rsi_min = df['rsi'].rolling(window=14).min()
        rsi_max = df['rsi'].rolling(window=14).max()
        df['stoch_rsi'] = 100 * ((df['rsi'] - rsi_min) / (rsi_max - rsi_min))

        # ATR для управления рисками
        high_low = df['High'] - df['Low']
        high_close = np.abs(df['High'] - df['Close'].shift())
        low_close = np.abs(df['Low'] - df['Close'].shift())
        true_range = np.maximum(np.maximum(high_low, high_close), low_close)
        df['atr'] = true_range.ewm(span=p['atr_period']).mean()

        # Объемный анализ
        df['volume_ma'] = df['Volume'].ewm(span=p['volume_ma']).mean()
        df['volume_ratio'] = df['Volume'] / df['volume_ma']

        # MACD с оптимизированными параметрами
        df['macd_fast'] = df['Close'].ewm(span=6).mean()
        df['macd_slow'] = df['Close'].ewm(span=13).mean()
        df['macd'] = df['macd_fast'] - df['macd_slow']
        df['macd_signal'] = df['macd'].ewm(span=5).mean()

        # Дополнительные индикаторы
        df['price_velocity'] = df['Close'].pct_change(3) * 100
        df['volatility'] = df['Close'].rolling(window=10).std() / df['Close'].rolling(window=10).mean() * 100

        return df

    def generate_advanced_signals(self, df, timeframe="5m"):
        """Генерация улучшенных торговых сигналов"""
        signals = []

        for i in range(10, len(df)):
            current = df.iloc[i]
            prev = df.iloc[i-1]

            # Основные сигналы
            ema_signal = self._ema_signal(current, prev)
            vwap_signal = self._vwap_signal(current, prev)
            rsi_signal = self._rsi_signal(current, prev)
            macd_signal = self._macd_signal(current, prev)
            volume_signal = self._volume_signal(current)
            price_action_signal = self._price_action_signal(df, i)

            # Взвешенная система оценки
            signal_score = 0
            confirmed_indicators = []

            weights = {
                'ema': 1.5, 'vwap': 1.2, 'rsi': 1.0,
                'macd': 1.3, 'volume': 0.8, 'price_action': 1.4
            }

            if ema_signal != 0:
                signal_score += ema_signal * weights['ema']
                confirmed_indicators.append("EMA")

            if vwap_signal != 0:
                signal_score += vwap_signal * weights['vwap']
                confirmed_indicators.append("VWAP")

            if rsi_signal != 0:
                signal_score += rsi_signal * weights['rsi']
                confirmed_indicators.append("RSI")

            if macd_signal != 0:
                signal_score += macd_signal * weights['macd']
                confirmed_indicators.append("MACD")

            if volume_signal != 0:
                signal_score += volume_signal * weights['volume']
                confirmed_indicators.append("VOL")

            if price_action_signal != 0:
                signal_score += price_action_signal * weights['price_action']
                confirmed_indicators.append("PA")

            # Генерация финального сигнала
            if signal_score >= 4.0 and len(confirmed_indicators) >= 4:
                signal_data = {
                    'timestamp': current.name,
                    'signal_type': 'BUY',
                    'price': float(current['Close']),
                    'strength': min(5, int(signal_score)),
                    'stop_loss': float(current['Close'] - 1.2 * current['atr']),
                    'take_profit': float(current['Close'] + 2.0 * current['atr']),
                    'indicators': ', '.join(confirmed_indicators),
                    'timeframe': timeframe
                }
                signals.append(signal_data)

            elif signal_score <= -4.0 and len(confirmed_indicators) >= 4:
                signal_data = {
                    'timestamp': current.name,
                    'signal_type': 'SELL',
                    'price': float(current['Close']),
                    'strength': min(5, int(abs(signal_score))),
                    'stop_loss': float(current['Close'] + 1.2 * current['atr']),
                    'take_profit': float(current['Close'] - 2.0 * current['atr']),
                    'indicators': ', '.join(confirmed_indicators),
                    'timeframe': timeframe
                }
                signals.append(signal_data)

        return signals

    def _ema_signal(self, current, prev):
        if (current['ema_fast'] > current['ema_slow'] and
            prev['ema_fast'] <= prev['ema_slow']):
            return 1
        elif (current['ema_fast'] < current['ema_slow'] and
              prev['ema_fast'] >= prev['ema_slow']):
            return -1
        return 0

    def _vwap_signal(self, current, prev):
        if (current['Close'] > current['vwap'] and
            prev['Close'] <= prev['vwap']):
            return 1
        elif (current['Close'] < current['vwap'] and
              prev['Close'] >= prev['vwap']):
            return -1
        return 0

    def _rsi_signal(self, current, prev):
        if current['rsi'] < 28 and prev['rsi'] >= 28 and current['stoch_rsi'] < 20:
            return 1.2
        elif current['rsi'] > 72 and prev['rsi'] <= 72 and current['stoch_rsi'] > 80:
            return -1.2
        elif current['rsi'] < 32 and prev['rsi'] >= 32:
            return 1
        elif current['rsi'] > 68 and prev['rsi'] <= 68:
            return -1
        return 0

    def _macd_signal(self, current, prev):
        if (current['macd'] > current['macd_signal'] and
            prev['macd'] <= prev['macd_signal'] and
            current['macd'] < 0):
            return 1.2
        elif (current['macd'] < current['macd_signal'] and
              prev['macd'] >= prev['macd_signal'] and
              current['macd'] > 0):
            return -1.2
        elif (current['macd'] > current['macd_signal'] and
              prev['macd'] <= prev['macd_signal']):
            return 1
        elif (current['macd'] < current['macd_signal'] and
              prev['macd'] >= prev['macd_signal']):
            return -1
        return 0

    def _volume_signal(self, current):
        if current['volume_ratio'] > 2.0:
            return 1.2
        elif current['volume_ratio'] > 1.5:
            return 0.8
        return 0

    def _price_action_signal(self, df, idx):
        if idx < 3:
            return 0

        current = df.iloc[idx]
        prev1 = df.iloc[idx-1]

        current_body = abs(current['Close'] - current['Open'])
        prev_body = abs(prev1['Close'] - prev1['Open'])

        # Bullish engulfing
        if (prev1['Close'] < prev1['Open'] and
            current['Close'] > current['Open'] and
            current['Open'] < prev1['Close'] and
            current['Close'] > prev1['Open'] and
            current['volume_ratio'] > 1.2):
            return 1.5

        # Bearish engulfing
        elif (prev1['Close'] > prev1['Open'] and
              current['Close'] < current['Open'] and
              current['Open'] > prev1['Close'] and
              current['Close'] < prev1['Open'] and
              current['volume_ratio'] > 1.2):
            return -1.5

        return 0

    def process_and_send_signals(self, symbol="BTC-USDT", timeframe="5m"):
        """Обработка и отправка сигналов"""
        # Получаем данные с OKX
        data = self.okx_client.get_candles(symbol, timeframe, 300)

        if data.empty:
            print("Нет данных для анализа")
            return []

        # Расчет индикаторов
        data = self.calculate_advanced_indicators(data)
        data = data.dropna()

        # Генерация сигналов
        signals = self.generate_advanced_signals(data, timeframe)

        # Отправка последних сигналов
        for signal in signals[-2:]:  # Только 2 последних сигнала
            if self.telegram_bot and self._should_send_signal(signal):
                # Добавляем анализ таймфреймов и доминацию
                timeframe_analysis, dominance = self.timeframe_analyzer.get_multi_timeframe_analysis(symbol)
                signal['timeframe_analysis'] = timeframe_analysis
                signal['btc_dominance'] = dominance

                self.telegram_bot.send_signal(signal)
                self.last_signal_time = signal['timestamp']

        return signals

    def _should_send_signal(self, signal):
        """Проверка, стоит ли отправлять сигнал"""
        if self.last_signal_time is None:
            return True

        # Не отправляем сигналы чаще чем раз в 10 минут
        time_diff = (signal['timestamp'] - self.last_signal_time).total_seconds()
        return time_diff > 600

def send_market_overview(telegram_bot, symbol="BTC-USDT"):
    """Отправка обзора рынка"""
    analyzer = OKXMultiTimeframeAnalyzer()
    timeframe_analysis, dominance = analyzer.get_multi_timeframe_analysis(symbol)

    # Получаем текущую цену
    okx_client = OKXClient()
    current_price = okx_client.get_ticker(symbol)

    message = f"""
📊 <b>ОБЗОР РЫНКА OKX</b>

💰 <b>Текущая цена {symbol}:</b> ${current_price:,.2f}
🏦 <b>Доминация BTC:</b> {dominance:.1f}%

<b>📈 МУЛЬТИТАЙМФРЕЙМ АНАЛИЗ:</b>
{timeframe_analysis}

⚡ <b>Источник:</b> OKX Spot
⏰ <b>Обновлено:</b> {datetime.now().strftime('%H:%M:%S')}
    """

    telegram_bot.send_message(message)

def live_monitoring(telegram_bot, symbol="BTC-USDT", interval="5m", check_interval=300):
    """Функция для живого мониторинга"""
    scalper = OKXScalpingAlgorithm(telegram_bot)

    # Приветственное сообщение
    send_market_overview(telegram_bot, symbol)

    telegram_bot.send_message(
        f"🚀 <b>OKX Scalping Bot активирован!</b>\n"
        f"Мониторинг: {symbol}\n"
        f"Таймфрейм: {interval}\n"
        f"Проверка каждые: {check_interval} сек\n"
        f"Время запуска: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
    )

    print(f"Запущен мониторинг {symbol} на OKX...")

    while True:
        try:
            print(f"Проверка сигналов... {datetime.now().strftime('%H:%M:%S')}")
            signals = scalper.process_and_send_signals(symbol, interval)

            if signals:
                print(f"Найдено сигналов: {len(signals)}")
            else:
                print("Сигналов не найдено")

            time.sleep(check_interval)

        except Exception as e:
            error_msg = f"❌ Ошибка мониторинга: {str(e)}"
            print(error_msg)
            telegram_bot.send_message(error_msg)
            time.sleep(60)

# ЗАПУСК АЛГОРИТМА
if __name__ == "__main__":
    # === НАСТРОЙКИ ===
    BOT_TOKEN = "8294892098:AAFX0Zzq9yN1on6UlID8f7vzif4dWR_7uWs"
    CHAT_ID = "381202205"
    SYMBOL = "BTC-USDT"  # Торговая пара на OKX
    TIMEFRAME = "5m"     # Таймфрейм для скальпинга

    # Инициализация
    telegram_bot = TelegramBot(BOT_TOKEN, CHAT_ID)

    # Тестовое сообщение
    print("Отправка тестового сообщения...")
    send_market_overview(telegram_bot, SYMBOL)

    # Тестирование алгоритма
    print("Тестирование алгоритма...")
    scalper = OKXScalpingAlgorithm(telegram_bot)
    test_signals = scalper.process_and_send_signals(SYMBOL, TIMEFRAME)

    print(f"Протестировано. Найдено сигналов: {len(test_signals)}")

    # Запуск живого мониторинга (раскомментируйте для постоянной работы)
    print("Запуск живого мониторинга...")
    live_monitoring(telegram_bot, SYMBOL, TIMEFRAME, 300)  # Проверка каждые 5 минут

Отправка тестового сообщения...
Тестирование алгоритма...
Протестировано. Найдено сигналов: 0
Запуск живого мониторинга...
Запущен мониторинг BTC-USDT на OKX...
Проверка сигналов... 05:51:10
Сигналов не найдено
